In [3]:
import logging
import os
import pickle
import time
import uuid
from typing import Final, List, Tuple

import cv2 as OpenCV
# import joblib
import numpy as np
# import trimesh
from numpy.linalg import norm
from scipy.cluster.vq import kmeans, vq
from scipy.spatial import Delaunay

In [4]:
class CalibrationError(Exception):
    def __init__(self, message):
        self.message = message

class IntrinsicParametersNotFoundError(Exception):
    def __init__(self, message):
        self.message = message

In [5]:
class Image:
    def __init__(self, img_id, rgb_image, gray_image, keypoints, descriptors, path):
        self.img_id: int = img_id
        self.unique_id: uuid = uuid.uuid4()
        self.rgb_image: Image = rgb_image
        self.gray_image: Image = gray_image
        self.keypoints: list[OpenCV.KeyPoint] = keypoints
        self.descriptors: np.ndarray = descriptors
        self.path: str = path
        self.similar_images: list[tuple[Image, float]] = []

    @property
    def length(self):
        return f"{len(self.keypoints)}" if len(self.keypoints) == len(self.descriptors) else f"{len(self.keypoints)}, {len(self.descriptors)}"
    
    def __repr__(self):
        return f"Image({self.img_id})"
    
    def __str__(self):
        return self.__repr__()
    
    def __eq__(self, other):
        return self.unique_id == other.unique_id
    
    def __hash__(self):
        return hash(self.img_id)
    
    def __getstate__(self):
        state = self.__dict__.copy()
        # Replace cv2.KeyPoint objects with tuples
        state['keypoints'] = [tuple(k.pt) + (k.size, k.angle, k.response, k.octave, k.class_id) for k in self.keypoints]
        return state
    
    def __setstate__(self, state):
        # Convert tuples back to cv2.KeyPoint objects
        state['keypoints'] = [OpenCV.KeyPoint(x, y, size, angle, response, octave, class_id) for x, y, size, angle, response, octave, class_id in state['keypoints']]
        self.__dict__ = state

class FeatureMatches:
    def __init__(self, image_one: Image, image_two: Image, matches: list[OpenCV.DMatch]):
        self.image_one: Image = image_one
        self.image_two: Image = image_two
        self.matches: list[OpenCV.DMatch] = matches

    def __repr__(self):
        return f"FeatureMatches({self.image_one}, {self.image_two} ---> {len(self.matches)})"

    def __getstate__(self):
        state = self.__dict__.copy()
        # Replace OpenCV.DMatch objects with tuples
        state['matches'] = [
            {'queryIdx': m.queryIdx, 'trainIdx': m.trainIdx, 'distance': m.distance} for m in self.matches
        ]
        return state
    
    def __setstate__(self, state):
        # Convert tuples back to OpenCV.DMatch objects
        state['matches'] = [
            OpenCV.DMatch(match['queryIdx'], match['trainIdx'], match['distance']) for match in state['matches']
        ]
        self.__dict__ = state
    
class Images:
    def __init__(self, images: list[Image], image_set_name: str):
        self.id = uuid.uuid4()
        self.images: list[Image] = images
        self.image_set_name: str = image_set_name
        self.feature_matches: list[FeatureMatches] = []

    def __len__(self):
        return len(self.images)


In [6]:
""" Step One: Read and Load Images
Inputs: 
- images_set_name: str

Outputs:
- gray_images: list[np.ndarray]

Main Functions:
1. read_images: read images from a folder
2. rgb_to_gray: convert a list of RGB images to grayscale

Utils Functions:
1. read_images_rbg: read an image from a path and convert it to RGB format
"""

def prepare_images(folder_path: str) -> Images:
    """ Read and load images """
    images: Images = Images([], folder_path.split("/")[-1])
    files: list[str] = filter(lambda file: ".jpg" in file, os.listdir(folder_path))
    for i, file in enumerate(files):
        image_path = f"{folder_path}/{file}"
        rgb_image = OpenCV.cvtColor(OpenCV.imread(image_path), OpenCV.COLOR_BGR2RGB)
        gray_image = OpenCV.cvtColor(rgb_image, OpenCV.COLOR_RGB2GRAY)
        images.images.append(Image(i, rgb_image, gray_image, [], [], image_path))
    return images

def dump_images_bak(images_file_path: str, images: Images) -> None:
    """ Dump images to a file """
    with open(images_file_path, "wb") as file:
        pickle.dump(images, file)

def load_images_bak(images_file_path: str) -> Images:
    """ Load images from a file """
    with open(images_file_path, "rb") as file:
        images = pickle.load(file)
    return images


In [7]:
"""Step Two: Feature Extraction
Inputs:
- gray_images: list[Image]
- SIFT: cv2.SIFT

Outputs:
- keypoints: list[np.ndarray]
- descriptors: list[np.ndarray]

Main Functions:
1. get_images_keypoints

Utils functions:
1. load_sift_features
2. dump_sift_features
3. convert_keypoints_to_tuples
4. convert_tuples_to_keypoints
"""

def compute_keypoints_descriptors(images: Image, SIFT: OpenCV.SIFT) -> None:
    """ Get keypoints and descriptors for each image in the list of images"""
    for img in images.images:
        keypoints: list[OpenCV.KeyPoint]
        descriptors: np.ndarray
        keypoints, descriptors = SIFT.detectAndCompute(img.gray_image, None)
        img.keypoints = keypoints
        img.descriptors = descriptors

In [8]:
""" Step Three: Image Matching
Inputs:
- descriptors: list[np.ndarray]

Outputs:
- centroids: np.ndarray
- variance: np.ndarray
- CLUSTER_COUNT: int
- matches_ids: list[list[tuple[int, float]]]

Main Functions:
1. match_images
2. get_matches_ids

Utils Functions:
1. load_image_matching
2. dump_image_matching
3. get_visual_words
4. get_frequency_vectors
5. get_tf_idf
6. search_matches
"""

def get_matches(images: Images) -> None:
    """ Match images using k-means clustering """
    all_descriptors = np.concatenate([image.descriptors for image in images.images])
    CLUSTER_COUNT: Final = 400
    ITER: Final = 2
    centroids, _ = kmeans(all_descriptors, CLUSTER_COUNT, ITER)
    matches_ids: list[list[tuple[int, float]]] =  get_matches_ids([image.descriptors for image in images.images], centroids, images.images)
    for i, image in enumerate(images.images):
        inner_list: list[Image, float] = [
            (images.images[match[0]], match[1]) for match in matches_ids[i]
        ]
        image.similar_images = inner_list


def get_visual_words(descriptors: list[np.ndarray], centroids: np.ndarray) -> list[np.ndarray]:
    visual_words = []
    for descriptor in descriptors:
        words, _ = vq(descriptor, centroids)
        visual_words.append(words)
    return visual_words


def get_frequency_vectors(visual_words: list[np.ndarray], CLUSTER_COUNT: int) -> np.ndarray:
    frequency_vectors = []
    for img_words in visual_words:
        histogram = np.zeros(CLUSTER_COUNT)
        for word in img_words:
            histogram[word] += 1
        frequency_vectors.append(histogram)
    return np.stack(frequency_vectors)


def get_tf_idf(frequency_vectors, IMAGES_COUNT) -> np.ndarray:
    df = np.sum(frequency_vectors > 0, axis = 0)
    idf = np.log(IMAGES_COUNT/df)
    return frequency_vectors * idf


def search_matches(i, top_clusters, tf_idf) -> list[tuple[int, float]]:
    """ Search for the top_clusters most similar images to the i-th image
    Args:
        i: the index of the image to search for similar images
        top_clusters: the number of similar images to return
        tf_idf: Term Frequency-Inverse Document Frequency
    Returns:
        A list of tuples, where each tuple contains the index of a similar image and the cosine similarity 
        between the i-th image and the similar image. The list is sorted by the cosine similarity in 
        descending order.
    """
    b = tf_idf
    a = tf_idf[i]
    b_subset = b[:tf_idf.shape[0]]
    cosine_similarity = np.dot(a, b_subset.T)/(norm(a) * norm(b_subset, axis=1))
    idx = np.argsort(-cosine_similarity)[:top_clusters]
    return list(zip(idx, cosine_similarity[idx]))


def get_matches_ids(descriptors, centroids, images_list) -> list[list[tuple[int, float]]]:
    """Returns: a list of lists, where each list contains the top 10 most similar images to the i-th image."""
    visual_words = get_visual_words(descriptors, centroids)
    frequency_vectors = get_frequency_vectors(visual_words, centroids.shape[0])
    """ tf_idf: Term Frequency-Inverse Document Frequency """
    tf_idf = get_tf_idf(frequency_vectors, len(images_list))
    return [
        search_matches(i, 10, tf_idf)
        for i in range(len(images_list))
    ]

In [9]:
"""Step Four: Feature Matching
Inputs:
- matches_ids: list[list[tuple[int, float]]]
- descriptors: list[np.ndarray]

Outputs:
- feature_matches: list[list[int, int, list[OpenCV.DMatch]]]

Main Functions:
1. data_feature_matching

Utils Functions:
1. feature_matching
2. dump_feature_matching
3. load_feature_matching
"""

def feature_matching(
        img_one_descriptors: np.ndarray, 
        img_two_descriptors: np.ndarray
    ) -> list[OpenCV.DMatch]:
    """ Match features between two images using Brute Force Matcher
    Args:
        img_id_one: the index of the first image
        img_id_two: the index of the second image
        descriptors: a list of descriptors of the images
    Returns:
        A list of OpenCV.DMatch objects.
    """
    matcher = OpenCV.BFMatcher()
    return matcher.match(img_one_descriptors, img_two_descriptors)

def data_feature_matching(images: Images) -> None:
    """ Match features between images using Brute Force Matcher
    Args:
        matchesIDs: a list of lists of tuples, where each tuple contains the index of a similar image and the cosine similarity 
            between the i-th image and the similar image. The list is sorted by the cosine similarity in 
            descending order.
        descriptors: a list of descriptors of the images
    Returns:
        A list of lists, where each list contains 
        the index of the first image, the index of the second image, 
        and a list of OpenCV.DMatch objects.
    """
    num_images: int = len(images.images)
    checked = np.zeros((num_images, num_images), dtype=int)
    # feature_matches_list: list[list[int, int, list[OpenCV.DMatch]]] = []
    for image in images.images:
        # logging.info(f"---------- START Matches for: {str(imageID)}")
        for matched_image, probability in image.similar_images:
            if ((checked[image.img_id][matched_image.img_id] == 0 or checked[matched_image.img_id][image.img_id] == 0) and image.img_id != matched_image.img_id and probability > 0.93):
                # start_time = time.time()
                images.feature_matches.append(FeatureMatches(image, matched_image, feature_matching(image.descriptors, matched_image.descriptors)))
                checked[image.img_id][matched_image.img_id], checked[matched_image.img_id][image.img_id] = 1, 1
                # logging.info(f"done [{i}/{len(matchesIDs[imageID])}] in {(time.time() - start_time):.4f}: {str(imageID)} - {str(matchedID)}")
        # Flush the log file force write to disk
        # logging.shutdown()


# def convert_matches_to_dicts(matches: list[OpenCV.DMatch]) -> list[dict[str, int, int, float]]:
#     """Convert a list of OpenCV.DMatch objects to a list of dictionaries to serialize."""
#     match_dicts = []
#     for match in matches:
#         match_dict = {'queryIdx': match.queryIdx, 'trainIdx': match.trainIdx, 'distance': match.distance}
#         match_dicts.append(match_dict)
#     return match_dicts


# def load_feature_matching(path: str) -> list[list[int, int, list[OpenCV.DMatch]]]:
#     """Load a list of lists of OpenCV.DMatch objects from a pickle file."""
#     with open(path, 'rb') as f:
#         feature_matches_dicts = pickle.load(f)
#     feature_matches = []
#     for match_dict in feature_matches_dicts:
#         matches = [
#             OpenCV.DMatch(
#                 match['queryIdx'], 
#                 match['trainIdx'], 
#                 match['distance']
#             ) for match in match_dict[2]
#         ]
#         feature_matches.append([match_dict[0], match_dict[1], matches])
#     return feature_matches


# def dump_feature_matching(path: str, feature_matches: list[list[int, int, list[OpenCV.DMatch]]]) -> None:
#     """Dump a list of lists of OpenCV.DMatch objects to a pickle file."""
#     matches_dicts = [
#         [
#             match[0],
#             match[1], 
#             convert_matches_to_dicts(match[2])
#         ] for match in feature_matches
#     ]
#     with open(path, 'wb') as f:
#         pickle.dump(matches_dicts, f)

In [10]:
"""Step Five: Camera Calibration
Inputs:
- None.

Outputs:
- k_matrix: np.ndarray

ToDo:
1- generate K_matrix.pickle for each camera using Chess board pattern.
"""

'Step Five: Camera Calibration\nInputs:\n- None.\n\nOutputs:\n- k_matrix: np.ndarray\n\nToDo:\n1- generate K_matrix.pickle for each camera using Chess board pattern.\n'

In [21]:
"""Step Six: Triangulation (3D Reconstruction)
Inputs:
- feature_matches_list: list[list[int, int, list[OpenCV.DMatch]]]
    -> A list of lists, where each list contains 
        the index of the first image, the index of the second image, 
        and a list of OpenCV.DMatch objects.
- K_matrix: np.ndarray
    -> The camera matrix of the camera used to take the images.

Outputs:
- point_cloud: list[np.ndarray]; each element is a 3D point.

Main Functions:
1. generate_point_cloud

Utils Functions:
1. triangulatePoints
"""

def triangulatePoints(P1, P2, pts1, pts2):
    """
    Triangulates the given matching points from two images using the given camera matrices.

    Parameters:
    P1 (numpy.ndarray): 3x4 camera matrix of the first image.
    P2 (numpy.ndarray): 3x4 camera matrix of the second image.
    pts1 (numpy.ndarray): Nx2 matrix containing the coordinates of matching points in the first image.
    pts2 (numpy.ndarray): Nx2 matrix containing the coordinates of matching points in the second image.

    Returns:
    numpy.ndarray: Nx3 matrix containing the triangulated 3D points.
    """
    pts4D = OpenCV.triangulatePoints(P1, P2, pts1.T, pts2.T)
    pts4D /= pts4D[3]
    return pts4D[:3].T


def generate_point_cloud(images: Images, K_matrix):
    """
    Generates a cloud of 3D points using triangulation from feature matches and camera calibration matrix.

    Parameters:
    feature_matches_list (list): List of feature matches between images.
    K_matrix (numpy.ndarray): 3x3 camera calibration matrix.

    Returns:
    numpy.ndarray: Nx3 matrix containing the cloud of 3D points.
    """
    point_cloud = []
    feature_matches_list = images.feature_matches
    for match in feature_matches_list:
        img_one = match.image_one
        img_two = match.image_two
        matches = match.matches
        pts1 = np.float32([img_one.keypoints[m.queryIdx].pt for m in matches])
        pts2 = np.float32([img_two.keypoints[m.trainIdx].pt for m in matches])
        E, _ = OpenCV.findEssentialMat(pts1, pts2, K_matrix)
        R1, R2, t = OpenCV.decomposeEssentialMat(E)

        for i in range(len(R1)):
            P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
            P2 = np.hstack((R1[i], t.reshape(3, 1)))
            # P2 = np.hstack((R1[i], t))
            pts_3d = triangulatePoints(K_matrix.dot(P1), K_matrix.dot(P2), pts1, pts2)
            point_cloud.append(pts_3d)
    return np.concatenate(point_cloud, axis=0)


In [12]:
print("Welcome ScanMate...")
image_set_name = "snow-man"
# 1. Load and prepare Images
if os.path.isfile(f"bak/{image_set_name}/images.pkl"):
    print(f"File [bak/{image_set_name}/sift-images.pkl] exists")
    print("Loading images from pickle file...")
    images: Images = load_images_bak(f"bak/{image_set_name}/images.pkl")
else:
    print(f"File [bak/{image_set_name}/images.pkl] does not exist")
    print("Loading images from images directory...")
    images: Images = prepare_images(f"images/{image_set_name}")
    print("Saving images to pickle file...")
    dump_images_bak(f"bak/{image_set_name}/images.pkl", images)
print("Images loaded successfully")

Welcome ScanMate...
File [bak/snow-man/sift-images.pkl] exists
Loading images from pickle file...
Images loaded successfully


In [13]:
# 2. Feature Extraction: SIFT
if os.path.isfile(f"bak/{image_set_name}/sift-features.pkl"):
    print(f"File [bak/{image_set_name}/sift-features.pkl] exists")
    images: Images = load_images_bak(f"bak/{image_set_name}/sift-features.pkl")
else:
    print("File [bak/{image_set_name}/sift-features.pkl] DO NOT exists")
    print("Extracting SIFT features...")
    sift = OpenCV.SIFT_create()
    compute_keypoints_descriptors(images, sift)
    dump_images_bak(f"bak/{image_set_name}/sift-features.pkl", images)
print("Feature Extraction: SIFT DONE...")

File [bak/snow-man/sift-features.pkl] exists
Feature Extraction: SIFT DONE...


In [14]:
print(len(images.images))
for i, image in enumerate(images.images):
    print(f"Image {i} has {len(image.keypoints)} keypoints and {len(image.descriptors)} descriptors")

171
Image 0 has 30209 keypoints and 30209 descriptors
Image 1 has 24060 keypoints and 24060 descriptors
Image 2 has 15784 keypoints and 15784 descriptors
Image 3 has 20075 keypoints and 20075 descriptors
Image 4 has 9188 keypoints and 9188 descriptors
Image 5 has 8359 keypoints and 8359 descriptors
Image 6 has 7740 keypoints and 7740 descriptors
Image 7 has 5260 keypoints and 5260 descriptors
Image 8 has 6567 keypoints and 6567 descriptors
Image 9 has 5377 keypoints and 5377 descriptors
Image 10 has 7142 keypoints and 7142 descriptors
Image 11 has 1003 keypoints and 1003 descriptors
Image 12 has 12652 keypoints and 12652 descriptors
Image 13 has 2439 keypoints and 2439 descriptors
Image 14 has 3968 keypoints and 3968 descriptors
Image 15 has 1067 keypoints and 1067 descriptors
Image 16 has 2312 keypoints and 2312 descriptors
Image 17 has 3024 keypoints and 3024 descriptors
Image 18 has 4884 keypoints and 4884 descriptors
Image 19 has 4276 keypoints and 4276 descriptors
Image 20 has 626

In [15]:
# 3. Image Matching
if os.path.isfile(f"bak/{image_set_name}/images-matched.pkl"):
    print(f"File [bak/{image_set_name}/images-matched.pkl] exists")
    images: Images = load_images_bak(f"bak/{image_set_name}/images-matched.pkl")
else:
    print(f"File [bak/{image_set_name}/images-matched.pkl] DO NOT exists")
    get_matches(images)
    dump_images_bak(f"bak/{image_set_name}/images-matched.pkl", images)
print("Done Image Matching Step...")

File [bak/snow-man/images-matched.pkl] exists
Done Image Matching Step...


In [16]:
# 4. Feature Matching
if os.path.isfile(f"bak/{image_set_name}/feature-matching-output.pkl"):
    print(f"File [bak/{image_set_name}/feature-matching-output.pkl] exists")
    images: Images = load_images_bak(f"bak/{image_set_name}/feature-matching-output.pkl")
else:
    print("File [bak/{image_set_name}/feature-matching-output.pkl] Do NOT exists")
    # logging.info('----> Processing {image_set_name}...')
    data_feature_matching(images)
    dump_images_bak(f"bak/{image_set_name}/feature-matching-output.pkl", images)

File [bak/snow-man/feature-matching-output.pkl] exists


In [19]:
# 5. Camera Calibration
print("Camera Calibration starts ....")
if not os.path.isfile("bak/snow-man/checker/K_matrix.pickle"):
    raise IntrinsicParametersNotFoundError("Intrinsic parameters not found")
print("File bak/snow-man/checker/K_matrix.pickle exists")
with open('bak/snow-man/checker/K_matrix.pickle', 'rb') as f:
    K_matrix = pickle.load(f)

Camera Calibration starts ....
File bak/snow-man/checker/K_matrix.pickle exists


In [22]:
# 6. Triangulation (3D reconstruction)
print("Triangulation starts ....")
points_cloud = generate_point_cloud(images, K_matrix)
np.savetxt("points_cloud.txt", points_cloud)

Triangulation starts ....


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
# 7. generate mesh
print("Generate mesh ....")
tri = Delaunay(points_cloud)
mesh = trimesh.Trimesh(points_cloud, tri.simplices)
mesh = mesh.simplify()

: 

In [ ]:
# 8. output .obj, .stl and .ply files
print("Generate mesh ....")
mesh.export(f"output/{image_set_name}/snow_man.obj")
mesh.export(f"output/{image_set_name}/snow_man.stl")
mesh.export(f"output/{image_set_name}/snow_man.ply")

: 